# Classify the tiles using pretrained tile2vec
In this notebook we are going to measure the performance of classifier on EuroSATallbands dataset.

In [20]:
import numpy as np
import os
import torch
from time import time
from torch.autograd import Variable

import sys
sys.path.append('../')
from src.tilenet import make_tilenet
from src.resnet import ResNet18
import pandas as pd
from pathlib import Path
from osgeo import gdal

## load the pretrained model

since the authors pretrained tile2vec on 4 channels (and provided the checkpoints on 4 channels) firstly we will evaluate its performance only on 4 first channels of our data

In [6]:
# Setting up model
in_channels = 4
z_dim = 512
cuda = torch.cuda.is_available()
# tilenet = make_tilenet(in_channels=in_channels, z_dim=z_dim)
# Use old model for now
tilenet = ResNet18()
if cuda: tilenet.cuda()

In [7]:
# Load parameters
model_fn = 'models/naip_trained.ckpt'
checkpoint = torch.load(model_fn)
tilenet.load_state_dict(checkpoint)
tilenet.eval()

ResNet(
  (conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [10]:
# Get data
train_csv = '/storage/EuroSATallBands/train.csv'

train_df = pd.read_csv(train_csv)

In [14]:
train_df.head()

,Filename,Label,ClassName
0,PermanentCrop/PermanentCrop_2401.tif,6,PermanentCrop
1,PermanentCrop/PermanentCrop_1006.tif,6,PermanentCrop
2,HerbaceousVegetation/HerbaceousVegetation_1025...,2,HerbaceousVegetation
3,SeaLake/SeaLake_1439.tif,9,SeaLake
4,River/River_1052.tif,8,River


In [21]:
base_dir = Path("/storage/EuroSATallBands/")

t0 = time()
n_tiles = len(train_df)
X = np.zeros((n_tiles, z_dim))

# this solution to iterate over examples is very suboptimal, one should use torch dataset
for index, row in train_df.iterrows():
    # read the tile from provided filepath
    
    tile_filepath = base_dir / row["Filename"]
    obj = gdal.Open(tile_filepath)
    img = obj.ReadAsArray().astype(np.float32)
    img = np.moveaxis(img, 0, -1)

    tile = img[:, :, :4] # model supports for now only 4 channels

    tile = np.moveaxis(tile, -1, 0)
    tile = np.expand_dims(tile, axis=0)

    tile = tile / 255
    # Embed tile
    tile = torch.from_numpy(tile).float()
    tile = Variable(tile)
    if cuda: tile = tile.cuda()
    z = tilenet.encode(tile)
    if cuda: z = z.cpu()
    z = z.data.numpy()
    X[index,:] = z


    if index % 100 == 0:
        print(f"embedded {index+1} images")

t1 = time()
print('Embedded {} tiles: {:0.3f}s'.format(n_tiles, t1-t0))

/home/tymek/.conda/envs/tile2vec/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


embedded 1 images
embedded 101 images
embedded 201 images
embedded 301 images
embedded 401 images
embedded 501 images
embedded 601 images
embedded 701 images
embedded 801 images
embedded 901 images
embedded 1001 images
embedded 1101 images
embedded 1201 images
embedded 1301 images
embedded 1401 images
embedded 1501 images
embedded 1601 images
embedded 1701 images
embedded 1801 images
embedded 1901 images
embedded 2001 images
embedded 2101 images
embedded 2201 images
embedded 2301 images
embedded 2401 images
embedded 2501 images
embedded 2601 images
embedded 2701 images
embedded 2801 images
embedded 2901 images
embedded 3001 images
embedded 3101 images
embedded 3201 images
embedded 3301 images
embedded 3401 images
embedded 3501 images
embedded 3601 images
embedded 3701 images
embedded 3801 images
embedded 3901 images
embedded 4001 images
embedded 4101 images
embedded 4201 images
embedded 4301 images
embedded 4401 images
embedded 4501 images
embedded 4601 images
embedded 4701 images
embe

In [22]:
y = train_df["Label"].to_numpy()

In [24]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [27]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)
rf = RandomForestClassifier()
rf.fit(X_tr, y_tr)
acc = rf.score(X_te, y_te)
print(f"Model accuracy: {acc}")

Model accuracy: 0.6868530020703933


not bad